# Code Specialization

To be fast, Julia needs to **specialize** code, that is compile specific native versions of the code. **The better the specialization the faster the code!** In the following we will investigate how Julia achieves good code specialization while retaining the power of generic programming.

## Just Ahead of Time (JAOT) Compilation

<p><img src="../imgs/from_source_to_native.png" alt="drawing" width="800"/></p>
 

**AST = Abstract Syntax Tree**

**IR = Intermediate Representation**

**SSA = Static Single Assignment**

**[LLVM](https://de.wikipedia.org/wiki/LLVM) = Low Level Virtual Machine**

## Specialization

**Julia specializes on the types of function arguments**, i.e. Julia compiles efficient machine code for the given input types, **when a function is called for the first time**.

If it is called again, the already existing machine code is reused, until we call the function with different input types.


In [ ]:
func(x,y) = 2x + y

In [ ]:
x = [1.2, 3.4, 5.6] # Vector{Float64}
y = [0.4, 0.7, 0.9] # Vector{Float64}

@time func(x,y);
@time func(x,y);

**First call:** compilation + running the code

**Second call:** running the code


In [ ]:
@time func(x,y);

If one of the input types changes, Julia compiles a new specialization of the function!


In [ ]:
typeof(x)

In [ ]:
x = [1, 3, 5]

In [ ]:
typeof(x)

In [ ]:
@time func(x,y); # Vector{Int64}, Vector{Float64}
@time func(x,y);

We now have two efficient native codes in the cache: one for all `Vector{Float64}` inputs and another one for `Vector{Int64}` as the first and `Vector{Float64}` as the second argument type.

In [ ]:
using MethodAnalysis

In [ ]:
methods(func)

In [ ]:
methodinstances(func)

## Introspection
#### (*But I really want to see what happens!*)

We can inspect the code at all transformation stages with a bunch of macros:

<img src="../imgs/julia_introspection_macros.png" width=350px>

In [ ]:
@macroexpand @time 3+3

In [ ]:
@code_lowered func(1.0,2.0)

In [ ]:
@code_typed func(1.0,2.0)

From the types of the input arguments, Julia has figured out all the intermediate types and replaced the generic functions `*` and `+` by specific implementations. This crucial process is known as **type inference** and its success is the basis for a good specialization (i.e. performant native code as a result). It will concern us in much more detail tomorrow.

In [ ]:
@code_llvm func(1.0,2.0)

We can remove the comments (lines starting with `;` using `debuginfo=:none`).


In [ ]:
@code_llvm debuginfo=:none func(1.0,2.0)

In [ ]:
@code_native debuginfo=:none syntax=:intel func(1.0,2.0)

Let's compare this to integer input.


In [ ]:
@code_native debuginfo=:none syntax=:intel func(1,2)

## How important is specialization?

Let's try to estimate the performance gain by specialization.

To prevent specialization, we deliberately throw away any useful type information and operate on a `Vector{Any}` that can literally store anything!

(This is qualitatively comparable to what Python does.)


In [ ]:
func(v) = 2*v[1] + v[2] # version of func that takes in a vector

In [ ]:
rand(2)

In [ ]:
Any[rand(), rand()]

In [ ]:
using BenchmarkTools

@btime func(v) setup=(v=rand(2));
@btime func(v) setup=(v=Any[rand(), rand()]);

**That's a huge slowdown!**


In [ ]:
@code_typed func(rand(2))

In [ ]:
@code_typed func(Any[rand(), rand()])

In [ ]:
# @code_native debuginfo=:none syntax=:intel func(rand(2))
# @code_native debuginfo=:none syntax=:intel func(Any[rand(), rand()])

## Types vs values

In high performance computing, compilation time (order of seconds or minutes) is typically neglectable compared to the actual time it takes to perform the computation (readily on the orders of hours/days/weeks). Therefore, we generally want to optimize for runtime efficiency even if this means that compilation time goes up by a reasonable amount.

**Julia specializes on input types and not values!**

Primarily it is **type information** that is used by the compiler to specialize code. (There are special techniques like, e.g., constant propagation and others that we are neglecting here.)

(Very) roughly speaking, the more information there is in *type space* (e.g. in type parameters) the higher the likelihood that the compiler produces fast and efficient code.

In [ ]:
A = rand(10,10);
B = rand(10,10);
@btime $A + $B;

In [ ]:
typeof(A)

In [ ]:
size(A)

In [ ]:
size(typeof(A)) # the size of A isn't type information

In [ ]:
using StaticArrays

In [ ]:
A = @SMatrix rand(10,10);
B = @SMatrix rand(10,10);

In [ ]:
typeof(A)

In [ ]:
size(typeof(A)) # the size of A is type information!

In [ ]:
@btime $A + $B;

**StaticArrays.jl**

```
============================================
    Benchmarks for 3×3 Float64 matrices
============================================
Matrix multiplication               -> 5.9x speedup
Matrix multiplication (mutating)    -> 1.8x speedup
Matrix addition                     -> 33.1x speedup
Matrix addition (mutating)          -> 2.5x speedup
Matrix determinant                  -> 112.9x speedup
Matrix inverse                      -> 67.8x speedup
Matrix symmetric eigendecomposition -> 25.0x speedup
Matrix Cholesky decomposition       -> 8.8x speedup
Matrix LU decomposition             -> 6.1x speedup
Matrix QR decomposition             -> 65.0x speedup
```

### Why not always use static arrays then?!

By putting more information in the type you are putting more stress on the compiler to optimize things.

Specifically, if static arrays are too big compile time can explode or the compiler might just give up and fall back to an inefficient default version.

Generally speaking, static arrays are only useful as small fixed-size arrays.

In [ ]:
# # should take (much) longer to compile and the speedup should be gone as well
# # if it isn't, increase N a little bit
# N = 50
# M = rand(N,N);
# Mstatic = SMatrix{N,N}(M);

# @btime $Mstatic + $Mstatic;
# @btime $M + $M;

### Dispatch and specialization

Having a reasonable amount of information encoded in the type domain isn't only useful to help the compiler (specialization) but also for dispatching to the most specific (and therefore hopfully most performant) method of a function.

**Types drive both specialization and multiple dispatch!**

In this sense, multiple dispatch is essentially the first step of the specialization process where Julia chooses between different implementations.

#### Example: Determinant of a 2x2 matrix

Let's say your task would be to write a function computing the determinant of a 2x2 matrix. How would you implement it?

Probably you'd say, well I know the formula for computing the determinant of a 2x2 matrix! Let's just implement it.


In [ ]:
det_2x2(X) = X[1,1] * X[2,2] - X[1,2] * X[2,1]

In [ ]:
M = [1 2; 3 4]

In [ ]:
det_2x2(M)

In [ ]:
@btime det_2x2(M);

Let's see how Julia's built-in `det` function compares to our algorithm:


In [ ]:
using LinearAlgebra

det(M)

In [ ]:
@btime det(M);

It's much slower!!

The reason isn't just that the compiler doesn't just know the size of the matrix from its type but also that [the code it considers](https://github.com/JuliaLang/julia/blob/release-1.8/stdlib/LinearAlgebra/src/generic.jl#L1544-L1550) (selected by the dispatch mechanism) is too general to compete with our implementation in `det_2x2`.

Let's now move the size information to the type domain and see how things change.

In [ ]:
using StaticArrays
S = @SMatrix [1 2; 3 4]

In [ ]:
@btime det($S);

Note that it is super faster because StaticArrays.jl provides [a hand-coded version](https://github.com/JuliaArrays/StaticArrays.jl/blob/master/src/det.jl#L10-L12), similar to our `det_2x2` above, which gets selected because of the size information in the type.

The (tiny) speed difference compared to our own `det_2x2` is only due to bounds checking and matrix vs linear indexing.

In [ ]:
det_2x2_optimized(X) = X[1] * X[4] - X[3] * X[2]
@btime det_2x2_optimized($M);

## Are explicit type annotations necessary? (think C or Fortran)

Note that Julia's type inference is powerful. Specifying types **is not** necessary for best performance!


In [ ]:
function my_function(x)
    y = rand()
    z = rand()
    x+y+z
end

function my_function_typed(x::Int)::Float64
    y::Float64 = rand()
    z::Float64 = rand()
    x+y+z
end

In [ ]:
@btime my_function(10);
@btime my_function_typed(10);

Annotating types explicitly can serve a purpose.

* Enforce conversions
* Very rarely: help the compiler infer types in tricky situations

However, more often than not it is an indication of suboptimal code design. (It also makes functions much less generic and reusable!)

# Core messages of this Notebook

* **A function is compiled when called for the first time** with a given set of argument types.
* The are **multiple compilation steps** which can be inspected through macros like `@code_warntype`.
* **Code specialization** based on the types of all of the input arguments is important for speed.
* Critical information can be moved to the **type domain** for better dispatch and specialization.
* In virtually all cases, **explicit type annotations are irrelevant for performance**.